# Make Frequency Dependence Figures

In [1]:
# Imports and funcs
%matplotlib widget
from brian2 import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import diagonal_sums as ds
import time
import datetime
import pickle

In [44]:
# Extract frequency and learning efficiency from all datasets
ndataset = 5
nsize = 2
nduty = 3
nfreq = 15
ngain = 2
nnodes = 500
frequency = np.zeros((ndataset,nsize*nduty*nfreq*ngain)) # reshape later
weight_profile = np.zeros((ndataset,nsize*nduty*nfreq*ngain,nnodes))
learning_efficiency = np.zeros((ndataset,nsize*nduty*nfreq))

# for frequency calcs
tmin = 20
tmax = 120
nbin = 40

for dataset in range(ndataset):
    with open('frequency_dependence_data'+str(dataset+1)+'.pkl', 'rb') as f:
        mylist = pickle.load(f)
    net_sizes = mylist[0]
    stim_duties = mylist[1]
    stim_frequencies = mylist[2]
    attractor_nodes = mylist[3]
    gain_record = mylist[4]
    time_record = mylist[5]
    spike_record = mylist[6]
    weight_record = mylist[7]
    attractor_size = len(attractor_nodes)
    attractor_index = {e:i for i,e in enumerate(attractor_nodes)} # lookup dictionary for getting attractor index from node number    
    
    for rec in range(nsize*nduty*nfreq*ngain):
        # Calculate frequency
        spikes_in_attractor = np.isin(spike_record[rec],attractor_nodes)
        z = spike_record[rec][spikes_in_attractor]
        spikes = np.ndarray(z.shape)
        for node in attractor_index:
            spikes[z==node] = attractor_index[node]
        t = time_record[rec][spikes_in_attractor]
        points_in_window = (t>tmin) & (t<tmax)
        t = t[points_in_window]
        spikes = spikes[points_in_window]
        real,timeline = np.histogram(t,bins=nbin,weights=np.cos(2.0*np.pi*spikes/attractor_size))
        imag,timeline = np.histogram(t,bins=nbin,weights=np.sin(2.0*np.pi*spikes/attractor_size))
        timeline = 0.5 * (timeline[:-1]+timeline[1:])
        cplx = real + 1.j * imag
        phase = np.unwrap(np.angle(cplx))
        p = np.polyfit(timeline,phase,1,w=np.absolute(cplx))
        frequency[dataset,rec] = 1000. * p[0] / (2.*np.pi)
        
        # Calculate weight profile
        weight_profile[dataset,rec] = ds.diagonal_sums(weight_record[rec])
    learning_efficiency[dataset,:] = 1./np.array(gain_record)

frequency = np.reshape(frequency,(ndataset,nsize,nduty,nfreq,ngain))
weight_profile = np.reshape(weight_profile,(ndataset,nsize,nduty,nfreq,ngain,nnodes))
learning_efficiency = np.reshape(learning_efficiency,(ndataset,nsize,nduty,nfreq))

mylist = [stim_duties, stim_frequencies, frequency, weight_profile, learning_efficiency]
with open('frequency_dependence_data_preprocessed.pkl', 'wb') as f:
    pickle.dump(mylist, f)

In [2]:
with open('frequency_dependence_data_preprocessed.pkl', 'rb') as f:
    mylist = pickle.load(f)
stim_duties = mylist[0]
stim_frequencies = mylist[1]
frequency = mylist[2]
weight_profile = mylist[3]
learning_efficiency = mylist[4]
ndataset,nsize,nduty,nfreq,ngain,nnodes = weight_profile.shape

In [3]:
print(ndataset,nsize,nduty,nfreq,ngain,nnodes,frequency.shape,np.std(frequency,0).shape)
frequency_mean = np.mean(frequency,0)
frequency_std = np.std(frequency,0)

print(frequency_mean.shape,frequency_std.shape)
print(learning_efficiency.shape)

5 2 3 15 2 500 (5, 2, 3, 15, 2) (2, 3, 15, 2)
(2, 3, 15, 2) (2, 3, 15, 2)
(5, 2, 3, 15)


In [82]:
frequency_mean = np.mean(frequency,0)
frequency_std = np.std(frequency,0)
learning_efficiency_mean = np.mean(learning_efficiency,0)
learning_efficiency_std = np.std(learning_efficiency,0)
weight_profile_mean = np.mean(weight_profile,0)
weight_profile_std = np.std(weight_profile,0)

figure(figsize=(12,4))
subplot(131)
#errorbar(stim_frequencies,np.reshape(frequency_mean[0,0,:,0],(-1,15)).T,np.reshape(frequency_std[0,0,:,0],(-1,15)).T,color='k',fmt='--')
plot(stim_frequencies,frequency_mean[1,0,:,0],linewidth=3,label='15cm, d=0.125')
plot(stim_frequencies,frequency_mean[1,1,:,0],linewidth=3,label='15cm, d=0.250')
plot(stim_frequencies,frequency_mean[1,2,:,0],linewidth=3,label='15cm, d=0.500')
plot(stim_frequencies,frequency_mean[0,0,:,0],label='1cm, d=0.125')
plot(stim_frequencies,frequency_mean[0,1,:,0],label='1cm, d=0.250')
plot(stim_frequencies,frequency_mean[0,2,:,0],label='1cm, d=0.500')
plot(stim_frequencies,frequency_mean[1,0,:,1],'tab:blue',linewidth=3,linestyle='dotted')
plot(stim_frequencies,frequency_mean[1,1,:,1],'tab:orange',linewidth=3,linestyle='dotted')
plot(stim_frequencies,frequency_mean[1,2,:,1],'tab:green',linewidth=3,linestyle='dotted')
plot(stim_frequencies,frequency_mean[0,0,:,1],'tab:red',linestyle='dotted')
plot(stim_frequencies,frequency_mean[0,1,:,1],'tab:purple',linestyle='dotted')
plot(stim_frequencies,frequency_mean[0,2,:,1],'tab:brown',linestyle='dotted')
#yscale('log')
xlabel('Driving Frequency (Hz)')
ylabel('Self-oscillation Frequency (Hz)')
axis([0,21,0,120])
#legend()

subplot(132)
learning_efficiency_norm = learning_efficiency_mean.copy()
learning_efficiency_norm[:,0,:] *= 2 # d=0.125 has a 2x handicap vs 0.25
learning_efficiency_norm[:,2,:] /= 2 # d=0.5 had a 2x advantage over 0.25
learning_efficiency_norm /= np.amax(learning_efficiency_norm)
plot(stim_frequencies,learning_efficiency_norm[1,0,:],linewidth=3,label='15cm, d=0.125')
plot(stim_frequencies,learning_efficiency_norm[1,1,:],linewidth=3,label='15cm, d=0.250')
plot(stim_frequencies,learning_efficiency_norm[1,2,:],linewidth=3,label='15cm, d=0.500')
plot(stim_frequencies,learning_efficiency_norm[0,0,:],label='1cm, d=0.125')
plot(stim_frequencies,learning_efficiency_norm[0,1,:],label='1cm, d=0.250')
plot(stim_frequencies,learning_efficiency_norm[0,2,:],label='1cm, d=0.500')
legend()
xlabel('Driving Frequency (Hz)')
ylabel('Relative Learning Efficiency $\eta$')
axis([0,21,0,1.05])

subplot(133)
#plot(np.arange(nnodes)-nnodes/2,np.reshape(weight_profile_mean[:,1,(0,14),0,:],(-1,nnodes)).T)
plot(np.arange(nnodes)-nnodes/2,weight_profile_mean[1,1,0,0,:],'tab:orange',label='15cm, f=1Hz')
plot(np.arange(nnodes)-nnodes/2,weight_profile_mean[1,1,14,0,:],'tab:orange',linestyle='dotted',label='15cm, f=20Hz')
plot(np.arange(nnodes)-nnodes/2,weight_profile_mean[0,1,0,0,:],'tab:purple',label='1cm, f=1Hz')
plot(np.arange(nnodes)-nnodes/2,weight_profile_mean[0,1,14,0,:],'tab:purple',linestyle='dotted',label='1cm, f=20Hz')
conduction_delay = 12.54e-3
k = -conduction_delay * frequency_mean[1,1,0,0] * nnodes
plot([k,k],[0,100],'tab:orange')
k = -conduction_delay * frequency_mean[1,1,14,0] * nnodes
plot([k,k],[0,100],'tab:orange',linestyle='dotted')
conduction_delay = 3.64e-3
k = -conduction_delay * frequency_mean[0,1,0,0] * nnodes
plot([k,k],[0,100],'tab:purple')
k = -conduction_delay * frequency_mean[0,1,14,0] * nnodes
plot([k,k],[0,100],'tab:purple',linestyle='dotted')
xlabel('Diagonal Number (k)')
ylabel('Sum of Diagonal')
legend()

tight_layout()
savefig('FIG2.pdf')

WARNING    <ipython-input-82-554b069f4a30>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure(figsize=(12,4))
 [py.warnings]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Manually check frequency fits to see if any failed
# Display spikes and fits in batches of 30 simulations
# There are 5 datasets and 180 simulations (6 batches) per dataset
# Read in one dataset using this cell, and change the value of start [0,30,60,90,120,150]
#  in the next cell to plot the fits
dataset = 4
with open('frequency_dependence_data'+str(dataset)+'.pkl', 'rb') as f:
    mylist = pickle.load(f)
net_sizes = mylist[0]
stim_duties = mylist[1]
stim_frequencies = mylist[2]
attractor_nodes = mylist[3]
gain_record = mylist[4]
time_record = mylist[5]
spike_record = mylist[6]
weight_record = mylist[7]
attractor_size = len(attractor_nodes)
attractor_index = {e:i for i,e in enumerate(attractor_nodes)} # lookup dictionary for getting attractor index from node number

In [17]:
nx=2
ny=15
figure(figsize=(4*nx,4*ny))

tmin = 20
tmax = 120
nbin = 40
batch = 30
start = 60 # [0,30,60,90,120,150]
for i in range(batch):
    rec = start + i
    spikes_in_attractor = np.isin(spike_record[rec],attractor_nodes)
    z = spike_record[rec][spikes_in_attractor]
    spikes = np.ndarray(z.shape)
    for node in attractor_index:
        spikes[z==node] = attractor_index[node]
    t = time_record[rec][spikes_in_attractor]
    points_in_window = (t>tmin) & (t<tmax)
    t = t[points_in_window]
    spikes = spikes[points_in_window]
    real,timeline = np.histogram(t,bins=nbin,weights=np.cos(2.0*np.pi*spikes/attractor_size))
    imag,timeline = np.histogram(t,bins=nbin,weights=np.sin(2.0*np.pi*spikes/attractor_size))
    timeline = 0.5 * (timeline[:-1]+timeline[1:])
    cplx = real + 1.j * imag
    phase = np.unwrap(np.angle(cplx))
    p = np.polyfit(timeline,phase,1,w=np.absolute(cplx))
    freq = 1000. * p[0] / (2.*np.pi)
    subplot(ny,nx,i+1)
    plot(t,spikes,'k.')
    plot(timeline,(attractor_size / (2.*np.pi) * (p[1] + timeline * p[0])) % attractor_size,'ro-')
    title(str(freq)+'Hz')

tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
print(learning_efficiency_mean)

[[[0.15098342 0.24664241 0.29146515 0.31335723 0.32485336 0.33111658
   0.32940172 0.32481615 0.3197728  0.31821241 0.31716387 0.31434491
   0.30889889 0.30844046 0.29677755]
  [0.27795564 0.53043011 0.68749167 0.79333562 0.85932178 0.88200297
   0.88788803 0.87450742 0.87566628 0.85275067 0.80482101 0.75673093
   0.70336424 0.66089827 0.63594343]
  [0.38209952 0.81233368 1.1855139  1.45661741 1.64778072 1.77141421
   1.81610989 1.83950786 1.83131524 1.7834398  1.68699729 1.55167352
   1.42881068 1.32412226 1.22873979]]

 [[0.13282677 0.22309402 0.25222483 0.27866336 0.29542505 0.28318891
   0.28234353 0.28091735 0.27487918 0.27844159 0.28312314 0.25441669
   0.25954007 0.25067932 0.24153547]
  [0.25190503 0.46470434 0.60380242 0.69353184 0.76488025 0.77603856
   0.77755913 0.79341181 0.79640219 0.71849639 0.72103348 0.64787197
   0.6087066  0.55179011 0.51573692]
  [0.34382899 0.7366392  1.05119868 1.29971028 1.43603158 1.54252715
   1.6844209  1.6067486  1.6554515  1.55783191 1.46448

In [6]:
1/0.87450742

1.1435008750411746

In [7]:
1/0.79341181

1.2603795247262577

In [1]:
2/1.6067486

1.2447498009333882

In [2]:
2/0.32481615

6.157329307671431

In [3]:
2/0.24664241

8.10890552034421

In [7]:
2/1.05706051

1.8920392740809135

In [8]:
print(learning_efficiency)

[[[[0.16634178 0.25453642 0.29173789 0.30421866 0.32507937 0.33074935
    0.32694764 0.33268356 0.32140615 0.31781502 0.31430325 0.31604938
    0.30260047 0.30260047 0.2962963 ]
   [0.27089947 0.53668763 0.69189189 0.79503106 0.83660131 0.85333333
    0.88275862 0.86486486 0.86486486 0.81528662 0.75739645 0.74418605
    0.69189189 0.65139949 0.62745098]
   [0.36702509 0.79503106 1.16363636 1.41436464 1.6        1.7414966
    1.82857143 1.79020979 1.81560284 1.72972973 1.6516129  1.55151515
    1.40659341 1.30612245 1.19626168]]

  [[0.1344891  0.22710135 0.25976662 0.2788671  0.30260047 0.27683158
    0.27683158 0.2994152  0.26155811 0.27284839 0.26155811 0.25453642
    0.2689782  0.24951267 0.24468339]
   [0.24468339 0.47672253 0.58850575 0.72316384 0.73988439 0.76646707
    0.72316384 0.82051282 0.8        0.66149871 0.69565217 0.63209877
    0.60520095 0.56140351 0.51405622]
   [0.34688347 0.74418605 1.03225806 1.33333333 1.42222222 1.6
    1.68421053 1.61006289 1.6516129  1.5515151

In [9]:
1/0.86486486

1.1562500065039063